In [ ]:
import os
import shapefile
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import MultipleLocator


def get_all_points_info(shp_path):
    ps = shapefile.Reader(shp_path)
    temp_list = []
    points_num = len(ps)
    for i in range(points_num):
        lon_lat = list(ps.shape(i).points[0])
        deformation = ps.record(i)[17:]
        coherence = ps.record(i)[1:2]
        mean_velocity = ps.record(i)[0:1]
        one_point_info = lon_lat + mean_velocity + coherence + deformation
        temp_list.append(one_point_info)
    all_points_info = np.array(temp_list)
    # print(all_points_info.shape)
    num = np.arange(0, points_num).reshape((-1, 1))
    all_points_info = np.concatenate([num, all_points_info], axis=1)
    # print(all_points_info.shape)
    date = []
    for i in ps.fields[18:]:
        date.append(i[0][2:])
    return all_points_info, date


def save_mean_vel(all_points_info, save_path):
    num = all_points_info[:, 0].reshape((-1, 1))
    lon = all_points_info[:, 1]
    lon = (lon - 0.000366).reshape((-1, 1))
    lat = all_points_info[:, 2].reshape((-1, 1))
    vel = all_points_info[:, 3].reshape((-1, 1))
    coh = all_points_info[:, 4].reshape((-1, 1))
    save_info = np.concatenate([num, lon, lat, vel, coh], axis=1)
    np.savetxt(save_path, save_info, fmt='%4f')


def plot_points_disp_in_one(cum_disp, date, num_list, y_lim, figsize=(10,5) ,flag=False, aspect=0.2):
    cum_disp = np.asarray(cum_disp, dtype=np.float32)
    points_disp = []

    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xlabel('date', fontsize=20)
    ax.set_ylabel('cumulative displacement (mm)', fontsize=20)
    ax.set_title('Point displacement', fontsize=25)
    ax.set_ylim(y_lim[0], y_lim[1])
    ax.set_aspect(aspect)
    y_major_locator=MultipleLocator(10)
    ax.yaxis.set_major_locator(y_major_locator)
    # ax.minorticks_on()
    ax.xaxis.grid(True, which='major')
    ax.xaxis.set_tick_params(rotation=30, labelsize=10)
    ax.yaxis.grid(True, which='major')
    ax.yaxis.set_tick_params(rotation=0, labelsize=10)
    ax.set_xmargin(0.02)
    for num in num_list:
        if num >= cum_disp.shape[0] or num < 0:
            print(f"Error: cannot find point {num}.")
        else:
            line = cum_disp[num, 5:]
            points_disp.append(list(line))
            ax.plot(date, line, label=str(num), marker='o')
            ax.xaxis.set_ticks(date[::2])
    ax.legend(loc='upper right', fontsize=15, ncol=2)
    fig.show()
    # save fig
    if flag:
        fig.savefig('cum_disp.png', dpi=600)
    # for i in points_disp:
    #     print('{}\n'.format(i))

## get points info(num, lon, lat, mean_velocity, coherence, deformation)

In [ ]:
shp_path = r"C:\thorly\Files\landslide\1_PS_processing\geocoding\1_PS_60_0.shp"
points_info, date = get_all_points_info(shp_path)

## prepare data for make_kmz_timeseries.py

In [ ]:
date = [int(d) for d in date]
first_line = [0, 0, 0] + date
first_line = np.asarray(first_line, np.float64).reshape((1, -1))
tmp = np.hstack((points_info[:, 1:4], points_info[:, 5:]))
save_data = np.vstack((first_line, tmp))

# save data
os.chdir(os.path.dirname(os.path.dirname(shp_path)))
np.savetxt('ts.txt', save_data, fmt='%4f')

## save velocity data(num, lon, lat, vel, coh)

In [ ]:
os.chdir(os.path.basename(os.path.basename(shp_path)))
save_mean_vel(points_info2, 'vel.txt')

## plot time-series displacement

In [ ]:
num_list = [1409, 1410]
plot_points_disp_in_one(points_info, date, num_list, y_lim=[-50, 50], figsize=(20,5) ,flag=False, aspect=0.1)